<a href="https://colab.research.google.com/github/shouvikcirca/PCB_Defect_Detection/blob/master/Resnet152v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import keras 
from keras.applications import ResNet152V2
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#######################

In [4]:
conv_base = ResNet152V2(weights='imagenet', include_top = False, input_shape=(300,300,3))

234553344/234545216 [==============================] - 3s 0us/step


In [5]:
from google.colab import drive
drive.mount("/content/gdrive/")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [ ]:
# conv_base.summary()

In [6]:
import pickle
# X = pickle.load(open(f'pickle_files/X2669.pkl', 'rb'))
# y = pickle.load(open(f'pickle_files/y2669.pkl', 'rb'))
X = pickle.load(open("gdrive/My Drive/X2669.pkl", 'rb'))
y = pickle.load(open("gdrive/My Drive/y2669.pkl", 'rb'))



X = X.permute(0,2,3,1).numpy()
y = y.numpy()

dataset_size = X.shape[0]

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [8]:
nos = X_train.shape[0]
nos

2135

In [9]:
((y==1).sum())/((y==0).sum()),((y_train==1).sum())/((y_train==0).sum()),((y_test==1).sum())/((y_test==0).sum())

(16.91275167785235, 15.811023622047244, 23.272727272727273)

In [10]:
y_test.shape, (y_test==1).sum(), (y_test==0).sum()

((534,), 512, 22)

In [11]:
# datagen = ImageDataGenerator(rescale=1./255, )
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [12]:
datagen.fit(X_train)

In [13]:
bs = 34
numit = NumpyArrayIterator(
    X_train, y_train, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

In [16]:
conv_base.predict(X_train[0].reshape(1,300,300,3)).shape


(1, 10, 10, 2048)

In [17]:
train_samples = np.zeros(shape=(nos,10,10,2048))
train_labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    train_samples[i*bs:(i+1)*bs] = f
    train_labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10, 2048)
(34, 10, 10,

In [ ]:
del X_train    # deleting X_train to make space

In [20]:
train_samples = train_samples.reshape(nos,10*10*2048)
train_samples.shape, train_labels.shape

((2135, 204800), (2135,))

In [21]:
nostest = X_test.shape[0]
nostest

534

In [22]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

In [23]:
datagentest.fit(X_test)

In [24]:
numitest = NumpyArrayIterator(
    X_test, y_test, datagentest, batch_size=nostest, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(nostest,10,10,2048))
test_labels = np.zeros(shape=(nostest))
for samples_batch,labels_batch in numitest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(534, 10, 10, 2048)


In [25]:
del X_test

In [26]:
test_samples = test_samples.reshape(nostest,10*10*2048)
test_samples.shape

(534, 204800)

In [27]:
from keras import models, layers, optimizers
model = models.Sequential()
model.add(layers.Dense(256, activation = 'relu', input_dim = 10*10*2048))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer = optimizers.Adam(lr = 1e-3),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
        


In [28]:
my_callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),

]

In [48]:
history = model.fit(train_samples, train_labels, epochs = 100, batch_size = 20,
                    validation_data = (test_samples, test_labels), callbacks = my_callbacks
)


Train on 2135 samples, validate on 534 samples
Epoch 1/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.0437 - accuracy: 0.9808 - val_loss: 0.2889 - val_accuracy: 0.9682
Epoch 2/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.0418 - accuracy: 0.9883 - val_loss: 0.3154 - val_accuracy: 0.9682
Epoch 3/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.0362 - accuracy: 0.9827 - val_loss: 0.3098 - val_accuracy: 0.9700
Epoch 4/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.1024 - accuracy: 0.9803 - val_loss: 0.5152 - val_accuracy: 0.9625
Epoch 5/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.1269 - accuracy: 0.9780 - val_loss: 0.5016 - val_accuracy: 0.9644
Epoch 6/100
2135/2135 [==============================] - 9s 4ms/step - loss: 0.1166 - accuracy: 0.9752 - val_loss: 0.3649 - val_accuracy: 0.9663


In [ ]:
# run = 1      22:100

In [49]:
preds = model.predict(test_samples, batch_size = 20)
predclass = np.array([1.0 if i>0.5 else 0.0 for i in preds])
(predclass == test_labels).sum()/534.0

0.9681647940074907

In [46]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_labels,predclass)

array([[ 10,  12],
       [  3, 509]])

In [50]:
confusion_matrix(test_labels,predclass)[0][0]/22, confusion_matrix(test_labels,predclass)[1][1]/512 

(0.3181818181818182, 0.99609375)

In [ ]:
test_labels.shape

(534,)

Validation Data

In [ ]:
xval = np.load('gdrive/My Drive/Copy of xtrain.npy')
yval = np.load('gdrive/My Drive/Copy of ytrain.npy')


In [ ]:
yval.shape[0]

298

In [ ]:
numitval = NumpyArrayIterator(
    xval, yval, datagen, batch_size=yval.shape[0], shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


val_samples = np.zeros(shape=(yval.shape[0],9,9,512))
val_labels = np.zeros(shape=(yval.shape[0]))
for samples_batch,labels_batch in numitval:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    val_samples[::] = g
    val_labels[:] = labels_batch
    break

(298, 9, 9, 512)


In [ ]:
val_samples = val_samples.reshape(yval.shape[0],9*9*512)

In [ ]:
val_preds = model.predict(val_samples, batch_size = 20)

In [ ]:
val_predclass = np.array([1.0 if i>0.5 else 0.0 for i in val_preds])

In [ ]:

(val_predclass == val_labels).sum()/298.0

0.9228187919463087

In [ ]:
confusion_matrix(val_labels,val_predclass)

array([[126,  23],
       [  0, 149]])

In [ ]:
confusion_matrix(val_labels,val_predclass)[0][0]/149, confusion_matrix(val_labels,val_predclass)[1][1]/149 

(0.8456375838926175, 1.0)

In [ ]:
# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

Saved model to disk


In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/